In [3]:
import pandas as pd
import numpy as np
from scipy.stats import weibull_min, pareto, expon
from datetime import timedelta

In [30]:
# Simular datos
N_DAYS = 365
np.random.seed(42)

df = pd.DataFrame({
    'fecha': pd.date_range(start = '2024-01-01', periods = N_DAYS, freq = 'D'),
    'estatus_falla': np.random.choice([0, 1], size = N_DAYS, p = [0.95, 0.05])
})

# Asegurar que la fecha sea de tipo datetime
df['fecha'] = pd.to_datetime(df['fecha'])

# Identificar las fechas donde el fallo ocurre
fechas_de_falla = df[df['estatus_falla'] == 1]['fecha']

# Calcular el tiempo de falla en dias
ttf_timedelta = fechas_de_falla.diff().dropna()

# convertir tiempo de falla a valor numerico en dias 
ttf_dias = ttf_timedelta.dt.days.rename('ttf_dias')

# Aseguramos tener un numero de datos adecuado para ajustar las distribuciones
if len(ttf_dias) < 10:
    print("Error: Insuficiente eventos de falla para ajustar una distribucion")

print(f"Numero de eventos de falla: {len(ttf_dias)}")
n = len(ttf_dias)

print("\n --- Ajustar distribuciones a la serie de tiempos de falla --- ")



Numero de eventos de falla: 15

 --- Ajustar distribuciones a la serie de tiempos de falla --- 


In [32]:
weibull_parametros = weibull_min.fit(ttf_dias, floc = 0) 
c_w, loc_w, scale_w = weibull_parametros

print(f"\n Parametros estimados de la distribucion Weibull (c: {c_w:.3f}, localizacion: {loc_w:.3f}, escala: {scale_w:.3f})")

L_w = weibull_min.logpdf(ttf_dias, *weibull_parametros).sum()
print(f"\n Log-Likelihood Weibull: {L_w:.3f}")

AIC_w = -2*L_w + 2*3
BIC_w = -2*L_w + 3*np.log(n)

print(f"\nAIC Weibull: {AIC_w:.3f}")
print(f"\nBIC Weibull: {BIC_w:.3f}")


 Parametros estimados de la distribucion Weibull (c: 1.003, localizacion: 0.000, escala: 21.563)

 Log-Likelihood Weibull: -61.044

AIC Weibull: 128.088

BIC Weibull: 130.212


In [33]:
expon_parametros = expon.fit(ttf_dias, floc = 0)
loc_e, escala_e = expon_parametros
print(f"\n Parametros estimados de la distribucion Exponencial ( localizacion: {loc_e:.3f}, escala = {escala_e:.3f}")

L_e = expon.logpdf(ttf_dias, *expon_parametros).sum()
print(f"\n Log-Likelihood Exponencial: {L_e:.3f}")

AIC_e = -2*L_e + 2*3
BIC_e = -2*L_e + 3*np.log(n)
print(f"\nAIC Exponencial: {AIC_e:.3f}")
print(f"\nBIC Exponencial: {BIC_e:.3f}")



 Parametros estimados de la distribucion Exponencial ( localizacion: 0.000, escala = 21.533

 Log-Likelihood Exponencial: -61.044

AIC Exponencial: 128.088

BIC Exponencial: 130.212


In [34]:
pareto_parametros = pareto.fit(ttf_dias, floc = 0)
b_p, loc_p, escala_p = pareto_parametros
print(f"\n Parametros estimados de la distribucion pareto (b: {b_p:.3f}, localizacion: {loc_p:.3f}, escala: {escala_p:.3f}")

L_p = pareto.logpdf(ttf_dias, *pareto_parametros).sum()
print(f"\n Log-Likelihood Pareto: {L_p:.3f}")

AIC_p = -2*L_p + 2*3
BIC_p = -2*L_p + 3*np.log(n)

print(f"\nAIC Pareto: {AIC_p:.3f}")
print(f"\nBIC Pareto: {BIC_p:.3f}")


 Parametros estimados de la distribucion pareto (b: 0.398, localizacion: 0.000, escala: 1.000

 Log-Likelihood Pareto: -66.448

AIC Pareto: 138.896

BIC Pareto: 141.020


In [57]:
# Prediccion
N_PREDICCIONES = 30
fechas_de_falla_ordenada = fechas_de_falla.sort_values()
ultima_fecha_de_falla = fechas_de_falla_ordenada.iloc[-1]
prediccion_intervalo_tf = weibull_min.rvs(
    c = c_w,
    loc = loc_w,
    scale = scale_w,
    size = N_PREDICCIONES
)
print(prediccion_intervalo_tf)
print(fechas_de_falla.sort_values())
print(type(fechas_de_falla_ordenada))
print(f"\n Ultima fecha de falla {ultima_fecha_de_falla}")


[20.34152274  7.36810501 18.77732688  3.63545084 14.16625264 16.41334877
  1.15797777  8.87351302  3.1313659   1.42378988 98.7431957   8.4153517
 35.73893523  6.36121918 24.6602936  30.75863377 19.52980587 13.77311171
 11.46711148  9.27569727 57.02241637 38.21825864 72.03115459  2.88002938
 28.2780271  59.884653    4.33323522  1.49614908 29.11220239 18.43268414]
1     2024-01-02
11    2024-01-12
34    2024-02-04
50    2024-02-20
69    2024-03-10
139   2024-05-19
140   2024-05-20
154   2024-06-03
226   2024-08-14
241   2024-08-29
247   2024-09-04
248   2024-09-05
261   2024-09-18
305   2024-11-01
313   2024-11-09
324   2024-11-20
Name: fecha, dtype: datetime64[ns]
<class 'pandas.core.series.Series'>

 Ultima fecha de falla 2024-11-20 00:00:00


In [55]:
# Crear la serie de fallas
prediccion_fechas = [ultima_fecha_de_falla + timedelta(days = int(interval))
                     for interval in prediccion_intervalo_tf]
print(prediccion_fechas)
len(prediccion_fechas)

[Timestamp('2024-11-23 00:00:00'), Timestamp('2024-11-26 00:00:00'), Timestamp('2024-11-23 00:00:00'), Timestamp('2024-11-24 00:00:00'), Timestamp('2024-11-27 00:00:00'), Timestamp('2024-11-24 00:00:00'), Timestamp('2025-01-07 00:00:00'), Timestamp('2024-11-21 00:00:00'), Timestamp('2024-12-06 00:00:00'), Timestamp('2024-12-01 00:00:00'), Timestamp('2025-02-14 00:00:00'), Timestamp('2024-11-22 00:00:00'), Timestamp('2024-11-30 00:00:00'), Timestamp('2025-02-02 00:00:00'), Timestamp('2025-01-02 00:00:00'), Timestamp('2024-12-26 00:00:00'), Timestamp('2024-11-26 00:00:00'), Timestamp('2024-11-24 00:00:00'), Timestamp('2024-12-13 00:00:00'), Timestamp('2025-01-15 00:00:00'), Timestamp('2024-12-07 00:00:00'), Timestamp('2024-12-08 00:00:00'), Timestamp('2024-11-27 00:00:00'), Timestamp('2024-12-21 00:00:00'), Timestamp('2024-11-24 00:00:00'), Timestamp('2024-11-28 00:00:00'), Timestamp('2024-12-01 00:00:00'), Timestamp('2024-12-05 00:00:00'), Timestamp('2024-11-26 00:00:00'), Timestamp('20

30

In [56]:
prediccion_df = pd.DataFrame({
    'prediccion_fecha_falla': prediccion_fechas,
    'dias_desde_ultimo_fallo': prediccion_intervalo_tf.astype(int)
})
print(prediccion_df)

   prediccion_fecha_falla  dias_desde_ultimo_fallo
0              2024-11-23                        3
1              2024-11-26                        6
2              2024-11-23                        3
3              2024-11-24                        4
4              2024-11-27                        7
5              2024-11-24                        4
6              2025-01-07                       48
7              2024-11-21                        1
8              2024-12-06                       16
9              2024-12-01                       11
10             2025-02-14                       86
11             2024-11-22                        2
12             2024-11-30                       10
13             2025-02-02                       74
14             2025-01-02                       43
15             2024-12-26                       36
16             2024-11-26                        6
17             2024-11-24                        4
18             2024-12-13      